# Solutions to checkpoint 5

In the following task, you'll continue working with the [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud/download) dataset from Kaggle. Before moving on to the tasks, you should load the dataset using Dask.

Please submit your solutions to the following tasks as a link to your jupyter notebook on Github.

In [0]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import roc_auc_score
import joblib
from dask.distributed import Client, progress
from dask_ml.model_selection import train_test_split
import dask.dataframe as dd
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [0]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:59093 Dashboard: http://127.0.0.1:59094/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


In [0]:
# This loads the data into Dask dataframe
df = dd.read_csv('data/creditcard.csv', dtype={'Time': 'float64'})

## 1. In this task, you'll train several machine learning models from scikit-learn using Dask as the backend of joblib. This time, you need to use all the variables except `Class` as your feature set. `Class` variable will be your target variable.

In [0]:
# This is our feature set
X = df.drop("Class", axis=1)

# This is our target variable
Y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Since our data can fit into memory
# we persist them to the RAM.
X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=3
    int64
      ...
      ...
      ...
Name: Class, dtype: int64
Dask Name: split, 3 tasks

In [0]:
lr = LogisticRegression()

with joblib.parallel_backend('dask'):
    lr.fit(X_train.compute(), y_train.compute())
    
preds_train = lr.predict(X_train.values.compute())
preds_test = lr.predict(X_test.values.compute())

print("Logistic regression training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Logistic regression test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Logistic regression training score is:  0.8679648305597004
Logistic regression test score is:  0.8846782089736235


In [0]:
gbc = GradientBoostingClassifier()

with joblib.parallel_backend('dask'):
    gbc.fit(X_train.compute(), y_train.compute())
    
preds_train = gbc.predict(X_train.values.compute())
preds_test = gbc.predict(X_test.values.compute())

print("Gradient boosting tree training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Gradient boosting tree test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Gradient boosting tree training score is:  0.92632027312039
Gradient boosting tree test score is:  0.9292868227268749


In [0]:
rfc = RandomForestClassifier()

with joblib.parallel_backend('dask'):
    rfc.fit(X_train.compute(), y_train.compute())
    
preds_train = rfc.predict(X_train.values.compute())
preds_test = rfc.predict(X_test.values.compute())

print("Random forest training score is: ", roc_auc_score(preds_train, y_train.values.compute()))
print("Random forest test score is: ", roc_auc_score(preds_test, y_test.values.compute()))

Random forest training score is:  0.9999648606505172
Random forest test score is:  0.9878482896721802


## 2. Compare the results of your models.

According to the results, the best performing model is the random forest. It's performance in both training and test sets are higher than those of the logistic regression and gradient boosting tree. However, we should note that we run these models using their default parameters. Ideally, one should do hyperparameter tuning by doing something like grid search or random search.